In [3]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [4]:
import dask.dataframe as dd
#!pip install fastparquet
#from fastparquet import ParquetFile

import re
import nltk
import json
import pandas as pd
import numpy as np
!pip install vaderSentiment
from nltk.corpus import wordnet 
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

nltk.download('wordnet')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)

#imports to get our parquet files aand dask files

     |████████████████████████████████| 133kB 3.3MB/s 
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [5]:
# Load pandas dataframe
def load_data(url):
    if url.endswith(".csv") or "csv" in url:
        df = pd.read_csv(url)
    else:
        df = None
    return df


In [6]:
rev_url = 'https://tally-ai-dspt3.s3.amazonaws.com/yelp-restaurants/rev_export.csv'

rev = load_data(rev_url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
rev.head()

,Unnamed: 0,review_id,reviews_stars,date,text
0,0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!"
1,1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The..."
2,2,uBWpaM4_PQ1NMB3N9nYkJA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price..."
3,3,EIcE_Xw9G5bDAY-XQ0Vosg,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garden medley with garlic and drunken noodles which should have lots of basil in it. The medley had no garlic and was sour. The drunken noodles had no basil and after we asked the rude owner for some basil he told us that its in there but he will bring us some raw basil. I think he must have been drunk to screw up 2 dishes that we have ordered 10 times before. He treats the customer like an idiot.
4,4,BXTxF_ijBmm5RPYELr-5RQ,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself is always pretty slow but the takeout is delicious. I really enjoy the soups. I have tried a number of entrees and they have been delicious. The lady dish I tried had sauteed pumpkin. It was something I have never tried but it was great!


In [8]:
rev.columns

Index(['Unnamed: 0', 'review_id', 'reviews_stars', 'date', 'text'], dtype='object')

In [9]:
rev = rev.drop(columns=["Unnamed: 0"])
rev.head()

,review_id,reviews_stars,date,text
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!"
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The..."
2,uBWpaM4_PQ1NMB3N9nYkJA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price..."
3,EIcE_Xw9G5bDAY-XQ0Vosg,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garden medley with garlic and drunken noodles which should have lots of basil in it. The medley had no garlic and was sour. The drunken noodles had no basil and after we asked the rude owner for some basil he told us that its in there but he will bring us some raw basil. I think he must have been drunk to screw up 2 dishes that we have ordered 10 times before. He treats the customer like an idiot.
4,BXTxF_ijBmm5RPYELr-5RQ,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself is always pretty slow but the takeout is delicious. I really enjoy the soups. I have tried a number of entrees and they have been delicious. The lady dish I tried had sauteed pumpkin. It was something I have never tried but it was great!


In [10]:
rev.dtypes

review_id         object
reviews_stars    float64
date              object
text              object
dtype: object

In [11]:
#rev['text'] = rev['text'].str


# Possible NLTK Text Cleaning to work around String

In [12]:
from nltk.tokenize import word_tokenize

def token(df):
  for x in reviews:
    tokens = word_tokenize(df['date'])
  words = [word for word in tokens if word.isalpha()]

In [13]:
df = rev

In [14]:
#df['cleaned_text'] = df['text']

In [15]:
#df['cleaned_text'] = df['cleaned_text'].replace(r'[^\w\s]','')

In [16]:
df.head()

,review_id,reviews_stars,date,text
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!"
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The..."
2,uBWpaM4_PQ1NMB3N9nYkJA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price..."
3,EIcE_Xw9G5bDAY-XQ0Vosg,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garden medley with garlic and drunken noodles which should have lots of basil in it. The medley had no garlic and was sour. The drunken noodles had no basil and after we asked the rude owner for some basil he told us that its in there but he will bring us some raw basil. I think he must have been drunk to screw up 2 dishes that we have ordered 10 times before. He treats the customer like an idiot.
4,BXTxF_ijBmm5RPYELr-5RQ,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself is always pretty slow but the takeout is delicious. I really enjoy the soups. I have tried a number of entrees and they have been delicious. The lady dish I tried had sauteed pumpkin. It was something I have never tried but it was great!


In [17]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
# df['cleaned_text'] = tokenizer.tokenize(df['text'])

In [18]:
# def customtokensize(text):
#     return tokenizer.tokenize(text)

# df['tokenized_text'] = df['text'].apply(customtokensize)
# df.head(2)

In [19]:
df.loc[df.isnull().any(axis=1)]

,review_id,reviews_stars,date,text
564707,NaN,NaN,NaN,NaN
1549812,iHdxK8lwkYn0rlNh-REuGQ,3.0,2014-12-26 08:32:06,NaN


In [20]:
df.loc[df.isna().any(axis=1)]

,review_id,reviews_stars,date,text
564707,NaN,NaN,NaN,NaN
1549812,iHdxK8lwkYn0rlNh-REuGQ,3.0,2014-12-26 08:32:06,NaN


In [21]:
df.shape

(3244807, 4)

In [22]:
df = df[df.isna().any(axis=1)==False]

In [23]:
## uncomment the below to test code on a smaller size dataframe
#df = df.sample(frac=0.05)

In [24]:
df2 = dd.from_pandas(df, npartitions=12)

In [25]:
df2.head()

,review_id,reviews_stars,date,text
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!"
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The..."
2,uBWpaM4_PQ1NMB3N9nYkJA,1.0,2012-03-07 21:51:40,"Moving to the Phoenix area was a shock to begin with and it has been difficult finding truly great Thai food establishments, especially in the far East Valley. However, of all the places I have tried, which is nearly 15 in just under 2 years, Papaya Thai on University Dr. in Mesa is the absolute WORST! \n\nI am a HUGE fan of all styles of Thai Food. I've traveled throughout Asia on more than one occassion, taught English in China and spent years seeking out authentic eateries in the Asian-based areas of this country's biggest Metropolitan cities. Thai food holds a special place in my heart, but never have I been as disappointed as I was with my experience at Papaya Thai!\n\nIf Papaya Thai was worth the words, I would gladly go into more detail about the atmosphere, the menu and the quality-to-value ratio, as I do with my other reviews... \n\nHowever, there's really not much to say about this dark and dirty hole-in-the-wall next to the Dollar Store. The service isn't worth the price..."
3,EIcE_Xw9G5bDAY-XQ0Vosg,1.0,2011-10-06 00:29:51,Usually ok but today we went and ordered garden medley with garlic and drunken noodles which should have lots of basil in it. The medley had no garlic and was sour. The drunken noodles had no basil and after we asked the rude owner for some basil he told us that its in there but he will bring us some raw basil. I think he must have been drunk to screw up 2 dishes that we have ordered 10 times before. He treats the customer like an idiot.
4,BXTxF_ijBmm5RPYELr-5RQ,4.0,2013-11-20 03:08:02,I have never dined in. The restaurant itself is always pretty slow but the takeout is delicious. I really enjoy the soups. I have tried a number of entrees and they have been delicious. The lady dish I tried had sauteed pumpkin. It was something I have never tried but it was great!


In [26]:
def customtokensize(text):
    return tokenizer.tokenize(text)

df2['tokenized_text'] = df2['text'].apply(customtokensize)
df2.head(2)

/usr/local/lib/python3.6/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('text', 'object'))

  warnings.warn(meta_warning(meta))


,review_id,reviews_stars,date,text,tokenized_text
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on!!","[Best, thai, food, in, Mesa, and, you, get, so, much, food, for, your, money, We, have, tried, almost, every, thai, place, in, the, area, and, think, Papaya, thai, is, by, far, the, best, and, always, ends, u, p, being, the, cheapest, too, Only, eating, here, for, thair, food, for, now, on]"
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the 3-starness of Papaya Thai, but it isn't necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. \n\nGiant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someone's head with the imaginary purse that I do not carry. \n\nNow onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they don't just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The...","[There, are, several, contributing, factors, to, the, 3, starness, of, Papaya, Thai, but, it, isn, t, necessarily, the, food, This, place, is, literally, a, block, away, from, my, house, and, I, have, ventured, frequently, because, of, its, proximity, reasonable, pricing, and, portioning, Giant, fish, tanks, with, some, cool, tropical, fish, are, strewn, about, but, then, there, is, this, tank, with, the, ugliest, eel, like, fish, and, catfish, It, makes, it, hard, for, me, to, eat, looking, at, them, so, I, opt, for, seats, facing, the, opposite, direction, I, also, feel, like, there, are, too, many, tables, for, the, amount, of, ...]"


# Sentiment Analysis that worked on sample

In [27]:
#load VADER
analyzer = SentimentIntensityAnalyzer()
#Add VADER metrics to dataframe


In [28]:
rev = df2
#remove emojis
rev['text'] = rev['text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE) 
rev['text'] = rev['text'].str.replace('[^\x1F-\x7F]+', '', flags=re.UNICODE)
rev['text'] = rev['text'].str.replace('\d+', '') #removes digits
rev.head(2)  # it seems like it's important to run .head() after any operation(s) that set a column value because dask seems to delay execution until the data is viewed and the operations stack up

,review_id,reviews_stars,date,text,tokenized_text
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on","[Best, thai, food, in, Mesa, and, you, get, so, much, food, for, your, money, We, have, tried, almost, every, thai, place, in, the, area, and, think, Papaya, thai, is, by, far, the, best, and, always, ends, u, p, being, the, cheapest, too, Only, eating, here, for, thair, food, for, now, on]"
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the -starness of Papaya Thai, but it isnt necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. Giant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someones head with the imaginary purse that I do not carry. Now onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they dont just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The price isnt ...","[There, are, several, contributing, factors, to, the, 3, starness, of, Papaya, Thai, but, it, isn, t, necessarily, the, food, This, place, is, literally, a, block, away, from, my, house, and, I, have, ventured, frequently, because, of, its, proximity, reasonable, pricing, and, portioning, Giant, fish, tanks, with, some, cool, tropical, fish, are, strewn, about, but, then, there, is, this, tank, with, the, ugliest, eel, like, fish, and, catfish, It, makes, it, hard, for, me, to, eat, looking, at, them, so, I, opt, for, seats, facing, the, opposite, direction, I, also, feel, like, there, are, too, many, tables, for, the, amount, of, ...]"


In [29]:
## this is how you get a dask.dataframe.shape, however compute() means that it has to scan the entire dataframe so it's very slow
#a = rev.shape
#a[0].compute(),a[1]

In [30]:
def get_compound(text):
  return analyzer.polarity_scores(text)['compound']

rev['compound'] = rev['text'].apply(get_compound)
rev.head(2)

/usr/local/lib/python3.6/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('text', 'float64'))

  warnings.warn(meta_warning(meta))


,review_id,reviews_stars,date,text,tokenized_text,compound
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on","[Best, thai, food, in, Mesa, and, you, get, so, much, food, for, your, money, We, have, tried, almost, every, thai, place, in, the, area, and, think, Papaya, thai, is, by, far, the, best, and, always, ends, u, p, being, the, cheapest, too, Only, eating, here, for, thair, food, for, now, on]",0.8555
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the -starness of Papaya Thai, but it isnt necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. Giant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someones head with the imaginary purse that I do not carry. Now onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they dont just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The price isnt ...","[There, are, several, contributing, factors, to, the, 3, starness, of, Papaya, Thai, but, it, isn, t, necessarily, the, food, This, place, is, literally, a, block, away, from, my, house, and, I, have, ventured, frequently, because, of, its, proximity, reasonable, pricing, and, portioning, Giant, fish, tanks, with, some, cool, tropical, fish, are, strewn, about, but, then, there, is, this, tank, with, the, ugliest, eel, like, fish, and, catfish, It, makes, it, hard, for, me, to, eat, looking, at, them, so, I, opt, for, seats, facing, the, opposite, direction, I, also, feel, like, there, are, too, many, tables, for, the, amount, of, ...]",0.4617


In [31]:
def get_neg(text):
  return analyzer.polarity_scores(text)['neg']

rev['neg'] = rev['text'].apply(get_neg)
rev.head(2)

/usr/local/lib/python3.6/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('text', 'float64'))

  warnings.warn(meta_warning(meta))


,review_id,reviews_stars,date,text,tokenized_text,compound,neg
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on","[Best, thai, food, in, Mesa, and, you, get, so, much, food, for, your, money, We, have, tried, almost, every, thai, place, in, the, area, and, think, Papaya, thai, is, by, far, the, best, and, always, ends, u, p, being, the, cheapest, too, Only, eating, here, for, thair, food, for, now, on]",0.8555,0.000
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the -starness of Papaya Thai, but it isnt necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. Giant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someones head with the imaginary purse that I do not carry. Now onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they dont just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The price isnt ...","[There, are, several, contributing, factors, to, the, 3, starness, of, Papaya, Thai, but, it, isn, t, necessarily, the, food, This, place, is, literally, a, block, away, from, my, house, and, I, have, ventured, frequently, because, of, its, proximity, reasonable, pricing, and, portioning, Giant, fish, tanks, with, some, cool, tropical, fish, are, strewn, about, but, then, there, is, this, tank, with, the, ugliest, eel, like, fish, and, catfish, It, makes, it, hard, for, me, to, eat, looking, at, them, so, I, opt, for, seats, facing, the, opposite, direction, I, also, feel, like, there, are, too, many, tables, for, the, amount, of, ...]",0.4617,0.057


In [32]:
def get_neu(text):
  return analyzer.polarity_scores(text)['neu']

rev['neu'] = rev['text'].apply(get_neu)
rev.head(2)

/usr/local/lib/python3.6/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('text', 'float64'))

  warnings.warn(meta_warning(meta))


,review_id,reviews_stars,date,text,tokenized_text,compound,neg,neu
0,Oa-sapLub0wRAzxRKVmtDQ,5.0,2017-02-23 03:10:38,"Best thai food in Mesa, and you get so much food for your money. We have tried almost every thai place in the area, and think Papaya thai is by far the best - and always ends u p being the cheapest too. Only eating here for thair food for now on","[Best, thai, food, in, Mesa, and, you, get, so, much, food, for, your, money, We, have, tried, almost, every, thai, place, in, the, area, and, think, Papaya, thai, is, by, far, the, best, and, always, ends, u, p, being, the, cheapest, too, Only, eating, here, for, thair, food, for, now, on]",0.8555,0.000,0.856
1,NawvdCkVIvhMO_eMVMdBhw,3.0,2010-12-04 00:38:28,"There are several contributing factors to the -starness of Papaya Thai, but it isnt necessarily the food. This place is literally a block away from my house, and I have ventured frequently because of its proximity, reasonable pricing, and portioning. Giant fish tanks with some cool tropical fish are strewn about, but then there is this tank with the ugliest eel-like fish and catfish. It makes it hard for me to eat looking at them, so I opt for seats facing the opposite direction. I also feel like there are too many tables for the amount of space. If the restaurant where ever full, I think I would knock into someones head with the imaginary purse that I do not carry. Now onto the food. I love that there are spiciness scales that I do not adhere too. I always ask extra, extra spicy, and they dont just throw a handful of red chile flakes into the dish that I order. Every curry dish I have ordered has tasted the way it should, not like coconut milk or peanuts. The price isnt ...","[There, are, several, contributing, factors, to, the, 3, starness, of, Papaya, Thai, but, it, isn, t, necessarily, the, food, This, place, is, literally, a, block, away, from, my, house, and, I, have, ventured, frequently, because, of, its, proximity, reasonable, pricing, and, portioning, Giant, fish, tanks, with, some, cool, tropical, fish, are, strewn, about, but, then, there, is, this, tank, with, the, ugliest, eel, like, fish, and, catfish, It, makes, it, hard, for, me, to, eat, looking, at, them, so, I, opt, for, seats, facing, the, opposite, direction, I, also, feel, like, there, are, too, many, tables, for, the, amount, of, ...]",0.4617,0.057,0.876


In [ ]:
def get_pos(text):
  return analyzer.polarity_scores(text)['pos']

rev['pos'] = rev['text'].apply(get_pos)
rev.head(2)

/usr/local/lib/python3.6/dist-packages/dask/dataframe/core.py:3073: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('text', 'float64'))

  warnings.warn(meta_warning(meta))


In [ ]:
  # rev['text_length'] = rev['text'].str.len()

In [ ]:
# rev['text_length'] = rev['text'].apply(len)
# rev.head(2)

In [ ]:
#columns = ['review_id', 'business_id', 'compound']
columns = ['review_id', 'compound', 'neg', 'neu', 'pos']

In [ ]:
df = rev[columns]

In [ ]:
# def sentiment_score(sentence):
#   # Create a SentimentIntensityAnalyzer object. 
#   sid_obj = SentimentIntensityAnalyzer()
#   # polarity_scores method of SentimentIntensityAnalyzer oject gives a sentiment dictionary. which contains pos, neg, neu, and compound scores. 
#   sentiment_dict = sid_obj.polarity_scores(sentence)
#   return sentiment_dict

# def get_sentiment(review_list):
#   all_sentiments = []
#   compounds = []
#   if len(review_list) > 0:
#     for review in review_list:
#       score = sentiment_score(review)
#       all_sentiments.append(score)
#   if len(all_sentiments) > 0:
#     for sentiment_dict in all_sentiments:
#       compound = sentiment_dict['compound']
#       compounds.append(compound)
# 	if len(compounds) > 0:
# 		avg_sentiment = sum(compounds) / len(compounds)
# 	else:
# 		avg_sentiment = None
# 	return avg_sentiment

# def get_subject_review_scores(review_list):
#     if len(review_list) > 0:
# 	    sub_sentiment_score = round((get_sentiment(review_list))*150)
# 		if sub_sentiment_score < 0:
# 		    sub_sentiment_score = 0
# 		else: 
# 			sub_sentiment_score = 75 #leaving at 75 to give neutral score
#     return sub_sentiment_score

# def sentimental_analysis(df):
#   # Create a SentimentIntensityAnalyzer object. 
#   sid_obj = SentimentIntensityAnalyzer()
#   # extract_subject_related_words
#   df['text'] = df['text'].apply(lambda x:" ".join(re.findall("[a-zA-Z]+", x)))
#   df['cleaned'] = df['text'].apply(tokenizer)
#   df['words_related_to_food'] = df['cleaned'].apply(related_to_food)
#   df['food_review_list'] = df[df['words_related_to_food'].map(len) > 1]['text'].tolist()
#   df['food_score'] = df['food_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_service'] = df['cleaned'].apply(related_to_service)
#   df['service_review_list'] = df[df['words_related_to_service'].map(len) > 1]['text'].tolist()
#   df['service_score'] = df['service_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_speed'] = df['cleaned'].apply(related_to_service)
#   df['speed_review_list'] = df[df['words_related_to_speed'].map(len) > 1]['text'].tolist()
#   df['speed_score'] = df['speed_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_price'] = df['cleaned'].apply(related_to_service)
#   df['price_review_list'] = df[df['words_related_to_price'].map(len) > 1]['text'].tolist()
#   df['price_score'] = df['price_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_ambience'] = df['cleaned'].apply(related_to_service)
#   df['ambience_review_list'] = df[df['words_related_to_ambience'].map(len) > 1]['text'].tolist()
#   df['ambience_score'] = df['ambience_review_list'].apply(get_subject_review_scores)
# 	df['words_related_to_experience'] = df['cleaned'].apply(related_to_service)
#   df['experience_review_list'] = df[df['words_related_to_experience'].map(len) > 1]['text'].tolist()
#   df['experience_score'] = df['experience_review_list'].apply(get_subject_review_scores)
#   return df

In [ ]:
rev.to_csv('sentiment.csv')
!cp rev.csv "drive/My Drive/"